In [1]:
import tensorflow as tf
from simple_model import TranslationModel
from utils import castom_layers
from utils.text_vectorise import custom_standardize
from keras.models import load_model

2025-07-24 20:26:14.155553: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-24 20:26:14.156208: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-24 20:26:14.159546: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-24 20:26:14.168263: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753381574.180139    4681 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753381574.18

In [2]:
model: TranslationModel = load_model('models/translation_model_final.keras',
                                    custom_objects={
                                        'DecoderMask': castom_layers.DecoderMask,
                                        'PadMask': castom_layers.PadMask,
                                        'custom_standardize': custom_standardize,
                                        'TranslationModel': TranslationModel
                                    }
                                )

E0000 00:00:1753381575.500839    4681 cuda_executor.cc:1228] INTERNAL: CUDA Runtime error: Failed call to cudaGetRuntimeVersion: Error loading CUDA libraries. GPU will not be used.: Error loading CUDA libraries. GPU will not be used.
W0000 00:00:1753381575.501652    4681 gpu_device.cc:2341] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
/home/toxa237/Projects/translation_example/.venv/lib/python3.12/site-packages/keras/src/saving/serialization_lib.py:730: UserWarning: Model 'translation_model' had a build config, but the model cannot be built automatically in `build_from_config(config)`. You should implement `def build_from_config(self, config)`, and you might also want to implement the method  that generates the config at savin

In [3]:
model.summary()

Model: "encoder_model"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_phrase        │ (None, 800)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embeding            │ (None, 800, 128)  │      9,856 │ input_phrase[0][… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pad_mask (PadMask)  │ (None, 1, 1, 800) │          0 │ input_phrase[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 800, 128)  │    527,488 │ embeding[0][0],   │
│ (MultiHeadAttentio… │                   │            │ embeding[0][0],   │
│                     │                   │            │ embeding[0][0],   │
│                     │                   │            │ pad_mask[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 800, 128)  │          0 │ multi_head_atten… │
│                     │                   │            │ embeding[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalization │ (None, 800, 128)  │        256 │ add[0][0]         │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 800, 128)  │     16,512 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 800, 128)  │          0 │ layer_normalizat… │
│                     │                   │            │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 800, 128)  │        256 │ add_1[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 554,368 (2.11 MB)

 Trainable params: 554,368 (2.11 MB)

 Non-trainable params: 0 (0.00 B)

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ decoder_input       │ (None, 800)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ emb_decoder         │ (None, 800, 128)  │      9,856 │ decoder_input[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_mask        │ (None, None,      │          0 │ decoder_input[0]… │
│ (DecoderMask)       │ None)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 800, 128)  │    527,488 │ emb_decoder[0][0… │
│ (MultiHeadAttentio… │                   │            │ emb_decoder[0][0… │
│                     │                   │            │ emb_decoder[0][0… │
│                     │                   │            │ decoder_mask[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_2 (Add)         │ (None, 800, 128)  │          0 │ multi_head_atten… │
│                     │                   │            │ emb_decoder[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 800, 128)  │        256 │ add_2[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embeding            │ (None, 800, 128)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 800, 128)  │    527,488 │ layer_normalizat… │
│ (MultiHeadAttentio… │                   │            │ embeding[0][0],   │
│                     │                   │            │ embeding[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_3 (Add)         │ (None, 800, 128)  │          0 │ layer_normalizat… │
│                     │                   │            │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 800, 128)  │        256 │ add_3[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 800, 128)  │     16,512 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_4 (Add)         │ (None, 800, 128)  │          0 │ layer_normalizat… │
│                     │                   │            │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 800, 128)  │        256 │ add_4[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 800, 77)   │      9,933 │ layer_normalizat… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,092,045 (4.17 MB)

 Trainable params: 1,092,045 (4.17 MB)

 Non-trainable params: 0 (0.00 B)

In [4]:
input_data = tf.constant(['Hello', 'my name is', 'where'], dtype=tf.string)

In [7]:
input_vectorized = model.text_vector(input_data)

# Створюємо <START> токени для декодера
batch_size = tf.shape(input_vectorized)[0]
decoder_input = tf.tile(model.text_vector(['<START>']), [batch_size, 1])

result_tokens = []

for i in range(1, 50):
    output_logits = model(input_vectorized, decoder_input, training=False)
    next_token = tf.argmax(output_logits[:, i], axis=-1)
    print(f'\r{i}/800: {next_token}', end='')
    decoder_input = tf.tensor_scatter_nd_update(decoder_input, [[k, i] for k, _ in enumerate(range(batch_size))], next_token)

    if tf.reduce_all(tf.equal(next_token, model.text_vector('<END>')[0])):
        break

# Перетворимо тензор [batch, seq] у текст
token_ids = decoder_input.numpy()  # прибираємо <START>
vocab = model.text_vector.get_vocabulary()
decoded_texts = []
for row in token_ids:
    chars = [vocab[i] for i in row]
    decoded_texts.append("".join(chars))

49/800: [44 48 48]

In [8]:
decoder_input, decoded_texts

(<tf.Tensor: shape=(3, 800), dtype=int64, numpy=
 array([[45, 48, 66, ...,  0,  0,  0],
        [45, 52, 68, ...,  0,  0,  0],
        [45, 62, 44, ...,  0,  0,  0]])>,
 ['§asi mea aa aa aaa aa aa raa aaa aa aa aa raaa aa ',
  '§eu saa aa aaa aa aa aaa aa aaa aa aaa aa aaa aa a',
  '§o sesi era aa aa aa aa raa aa aa raaa aa aa aa ra'])

In [ ]:
decoder_input

In [17]:
 tf.tensor_scatter_nd_update(decoder_input, [[0, 1], [1, 1], [2, 1]], [1, 2, 4])

<tf.Tensor: shape=(3, 800), dtype=int64, numpy=
array([[45,  1,  0, ...,  0,  0,  0],
       [45,  2,  0, ...,  0,  0,  0],
       [45,  4,  0, ...,  0,  0,  0]])>

In [11]:
tf.argmax(output_logits[:, i], axis=-1)

<tf.Tensor: shape=(3,), dtype=int64, numpy=array([52, 52, 52])>

In [10]:
tf.argmax(output_logits, axis=-1)

<tf.Tensor: shape=(3, 800), dtype=int64, numpy=
array([[52, 29, 29, ..., 48, 48, 48],
       [52, 52, 29, ..., 60, 60, 60],
       [52, 29, 29, ..., 48, 48, 48]])>

In [16]:
[[k, i] for k, _ in enumerate(range(batch_size))]

[[0, 0], [1, 0], [2, 0]]